# Imports & settings

In [ ]:
import sys

from dateutil.relativedelta import relativedelta
import time, datetime
import xlrd
import os, getpass, requests, sys, asyncio, aiohttp, time, logging,datetime
import functools
from pandas.io.json import json_normalize
import pandas as pd
import logging
import os
cwd = os.getcwd()
folder_loc = str (cwd +"/files/")


# Logging

In [ ]:
logfile = str(folder_loc + 'asynclog' + ".log")
logging.basicConfig(level=logging.DEBUG, filename=logfile, format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p')
logging.warning("Process started")

# API Credentials

In [ ]:
apiKey = os.environ.get('YourAPIKey')
Bearer= 'Bearer ' + apiKey
headers = {"Authorization": Bearer}
url = "YourURL?page={}&per_page={}"
per_page = 100

## Add Callback Function

In [ ]:
def ReturnResult(future):
    logging.warning("Process Complete")
    print(time.strftime('%y-%m-%d %H:%M'))
    print(future.done())
    
    try:
        print(len(results))
    except:
        pass

# API Call

In [ ]:
async def Get_One_Page(session: aiohttp.ClientSession(), url: str, page: int, sem: asyncio.Semaphore):
    async with sem:
        if sem.locked():
            print("Concurrency limit reached, waiting ..."+str(page))
#             await asyncio.sleep(1)
                
        try:
            ur = await session.get(url.format(page, per_page), headers={"Authorization": Bearer})
            r = await ur.json()
            return r
        except:
            print(sys.exc_info()[1])



async def Get_All_Pages(url: str, pages: int):
    tasks = []
    semaphore = asyncio.Semaphore(9)
    async with aiohttp.ClientSession() as session: #open session    
        try:
            results = await asyncio.gather(*[Get_One_Page(session, url, num, semaphore) for num in range(1,pages+1)])
        except:
            print(sys.exc_info()[1])
            
    return results

        

# Run Process

In [ ]:
loop = asyncio.get_event_loop()
loop.set_debug(True)
# session = aiohttp.ClientSession()

In [ ]:
future = asyncio.run_coroutine_threadsafe(Get_All_Pages(url, 20), loop)
future.add_done_callback(ReturnResult)

# JSON Result
returns a list of JSONs - you can flatten to a df

In [ ]:
pages = future.result()
